## Tensor Board


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from skimage import io, color
from skimage import exposure
from time import time

from keras import backend
from keras.models import Sequential
from keras import models
from keras import layers
from keras import metrics
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model
from keras import backend as K
from keras.callbacks import TensorBoard
from keras import metrics

Using TensorFlow backend.


In [2]:
from keras.datasets import mnist
from keras.utils import np_utils

In [3]:
(x_train, y_train),  (x_test, y_test) = mnist.load_data()
img_rows, img_cols = x_train.shape[1:]

In [ ]:
plt.imshow(x_train[0], cmap=plt.get_cmap('gray'))

In [4]:
# reshape para meter la dimensión del canal
x_train_ = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test_ = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
                           
input_shape = (img_rows, img_cols, 1)
# normalizamos
x_train_ = x_train_.astype('float32')/255
x_test_ = x_test_.astype('float32')/255

nb_classes = 10
y_train_ = np_utils.to_categorical(y_train, nb_classes)
y_test_ = np_utils.to_categorical(y_test, nb_classes)

In [33]:
# limpiar sesion (higiene en nombres)
backend.clear_session()

nb_filters = 32 # queremos que aprenda 32 filtros distintos
kernel_size = (3,3)
pool_size=(2,2)
nb_classes = 10

model = Sequential()
model.add(Conv2D(filters = nb_filters, input_shape = input_shape, kernel_size = kernel_size, padding='same', activation = 'relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Flatten())
model.add(Dense(32, activation = 'relu'))
model.add(Dense( 10, activation = 'softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                200736    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 201,386
Trainable params: 201,386
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=[metrics.categorical_accuracy])

In [36]:

tensorboard = TensorBoard(
    log_dir = 'logs/',
    histogram_freq = 1
)

model.fit(x_train_, y_train_, epochs=5, verbose=1, callbacks=[tensorboard], validation_split=.2)
# callback para que visualice en tensor boaard

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 16s 338us/step - loss: 0.5329 - categorical_accuracy: 0.8554 - val_loss: 0.2894 - val_categorical_accuracy: 0.9157
Epoch 2/5
48000/48000 [==============================] - 17s 353us/step - loss: 0.2741 - categorical_accuracy: 0.9185 - val_loss: 0.2307 - val_categorical_accuracy: 0.9323
Epoch 3/5
48000/48000 [==============================] - 17s 355us/step - loss: 0.2280 - categorical_accuracy: 0.9323 - val_loss: 0.2055 - val_categorical_accuracy: 0.9407
Epoch 4/5
48000/48000 [==============================] - 17s 345us/step - loss: 0.1966 - categorical_accuracy: 0.9416 - val_loss: 0.1852 - val_categorical_accuracy: 0.9451
Epoch 5/5
48000/48000 [==============================] - 17s 351us/step - loss: 0.1736 - categorical_accuracy: 0.9483 - val_loss: 0.1638 - val_categorical_accuracy: 0.9518


con la red entrenando invocamos tensorboard:

```
$ tensorboard --logdir = ./logs
```

Que nos devuelve la url del visor (localhost:6006)

## Otra arquitectura con API funcional

Vamos a aplicar convoluciones con kernels de distinto tamaño en paralelo

In [48]:
backend.clear_session()


tensorboard = TensorBoard(
    log_dir = 'logs/',
    histogram_freq = 1
)

#input layer
input_layer = layers.Input(shape = input_shape, dtype='float32', name='input_layer')

# kernels de  distinto tamaño
# 3,3
conv_33_inst = layers.Conv2D(filters = nb_filters, input_shape = input_shape, kernel_size = (3,3), padding='same', activation = 'relu', name='conv_33')
mxpool_33_inst = layers.MaxPooling2D(pool_size=pool_size, name='mxpool_33')
conv_33 = conv_33_inst(input_layer)
mxpool_33 = mxpool_33_inst(conv_33)

# 5,5
conv_55_inst = layers.Conv2D(filters = nb_filters, input_shape = input_shape, kernel_size = (5,5), padding='same', activation = 'relu', name='conv_55')
mxpool_55_inst = layers.MaxPooling2D(pool_size=pool_size, name='mxpool_55')
conv_55 = conv_55_inst(input_layer)
mxpool_55 = mxpool_55_inst(conv_55)

# 7,7 A estas alturas ya debería haber hecho un bucle for
conv_77_inst = layers.Conv2D(filters = nb_filters, input_shape = input_shape, kernel_size = (7,7), padding='same', activation = 'relu', name='conv_77')
mxpool_77_inst = layers.MaxPooling2D(pool_size=pool_size, name='mxpool_77')
conv_77 = conv_77_inst(input_layer)
mxpool_77 = mxpool_77_inst(conv_77)


# concatenate 
#concat_layer = layers.concatenate([mxpool_33, mxpool_55, mxpool_77], name = 'concat')
# (pero podríamos sumar)
concat_layer = layers.add([mxpool_33, mxpool_55, mxpool_77], name = 'concat')
# flatten
flatten_layer_inst = layers.Flatten(name='flatten')
flatten_layer = flatten_layer_inst(concat_layer)

# Densas de salida
dense_1_inst = layers.Dense(32,activation='relu', name = 'dense1')
dense_1 = dense_1_inst(flatten_layer)

out_inst = layers.Dense(10, activation='softmax', name='out')
out = out_inst(dense_1)

model = models.Model(input_layer, out)

model.summary()



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv_33 (Conv2D)                (None, 28, 28, 32)   320         input_layer[0][0]                
__________________________________________________________________________________________________
conv_55 (Conv2D)                (None, 28, 28, 32)   832         input_layer[0][0]                
__________________________________________________________________________________________________
conv_77 (Conv2D)                (None, 28, 28, 32)   1600        input_layer[0][0]                
__________________________________________________________________________________________________
mxpool_33 

In [49]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=[metrics.categorical_accuracy])
model.fit(x_train_, y_train_, epochs=5, verbose=1, callbacks=[tensorboard], validation_split=.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 55s 1ms/step - loss: 0.4323 - categorical_accuracy: 0.8734 - val_loss: 0.2287 - val_categorical_accuracy: 0.9337
Epoch 2/5
48000/48000 [==============================] - 54s 1ms/step - loss: 0.2007 - categorical_accuracy: 0.9404 - val_loss: 0.1546 - val_categorical_accuracy: 0.9558
Epoch 3/5
48000/48000 [==============================] - 53s 1ms/step - loss: 0.1366 - categorical_accuracy: 0.9600 - val_loss: 0.1143 - val_categorical_accuracy: 0.9682
Epoch 4/5
48000/48000 [==============================] - 55s 1ms/step - loss: 0.1047 - categorical_accuracy: 0.9694 - val_loss: 0.1038 - val_categorical_accuracy: 0.9706
Epoch 5/5
48000/48000 [==============================] - 56s 1ms/step - loss: 0.0858 - categorical_accuracy: 0.9753 - val_loss: 0.0836 - val_categorical_accuracy: 0.9762


# CNN's con autoencoder

In [24]:
# limpiar sesion (higiene en nombres)
backend.clear_session()
pool_size=(2,2)
nb_classes = 10


input_img = Input(shape=(28,28,1))
x = layers.Conv2D(filters = 16, kernel_size = (3,3), padding='same', activation = 'relu')(input_img)
x = layers.MaxPooling2D(pool_size=pool_size)(x)
x = layers.Conv2D(filters = 8, kernel_size = (3,3), padding='same', activation = 'relu')(x)
encoded = layers.MaxPooling2D(pool_size=pool_size)(x)

x = layers.Conv2D(filters=8, kernel_size=(3,3), padding='same', activation='relu')(encoded)
x = layers.UpSampling2D(size=(2,2))(x)
x = layers.Conv2D(filters = 16, kernel_size = (3,3), padding='same', activation = 'relu')(x)
x = layers.UpSampling2D(size=(2,2))(x)

decoded = layers.Conv2D(filters=1, kernel_size=(3,3), padding='same', activation='relu')(x)

autoencoder = models.Model(input_img, decoded)
autoencoder.summary()
autoencoder.compile(optimizer = 'adadelta', loss='binary_crossentropy')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 8)           584       
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 8)         0         
__________

In [25]:
autoencoder.fit(x_train_, y_train_, epochs=5, validation_split=.2, verbose=1)

ValueError: Error when checking target: expected conv2d_5 to have 4 dimensions, but got array with shape (60000, 10)

## Text and CNN's

Dataset : https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences

In [28]:
import pandas as pd
filepath_dict = {
    'yelp':'../data/sentiment labelled sentences/yelp_labelled.txt',
    'amazon':'../data/sentiment labelled sentences/amazon_cells_labelled.txt',
    'imdb':'../data/sentiment labelled sentences/imdb_labelled.txt'
}

In [29]:

df_list = []
for source, filepath in filepath_dict.items():
    df = pd.read_csv(filepath, names=['sentence', 'label'], sep='\t')
    df['source'] = source  # Add another column filled with the source name
    df_list.append(df)

df = pd.concat(df_list)
df.head()
sentences_train = 

,sentence,label,source
0,Wow... Loved this place.,1,yelp
1,Crust is not good.,0,yelp
2,Not tasty and the texture was just nasty.,0,yelp
3,Stopped by during the late May bank holiday of...,1,yelp
4,The selection on the menu was great and so wer...,1,yelp


Vamos a mapear cada palabra a  un número, de forma que cada frase es un un vector. La dimensión del vector viene dado por la frase más larga, de modo que  se hace un padding para rellenar las dimensiones que faltan.

In [31]:
from keras.preprocessing.text import Tokenizer # mapeo de palabras
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sentences_train)

x_train = tokenizer.text_to_sequences(sequences_train)
x_test = tokenizer.text_to_sequences(sequences_test)

NameError: name 'sentences_train' is not defined